# 9. 모델 프리징(Model Freezing)

전이 학습 중 잘 학습 된 모델을 가져와 우리 연구에 사용할 수 있다. 데이터가 유사한 경우에는 추가적인 전체 학습 없이도 좋은 성능이 나올 수 있다. 따라서 피쳐 추출에 해당하는 합성곱 층의 변수를 업데이트 하지 않고 분류 파트에 해당하는 fully connected layer의 변수만 업데이트 할 수 있는데 이 때 변수가 업데이트 되지 않게 변수를 얼린다고 하여 이를 프리징(Freezing)이라고 한다.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch.nn as nn
import torch.optim as optim

In [2]:
# GPU vs CPU
# 현재 가능한 장치를 확인한다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## 9.2 CIFAR10 데이터 불러오기

In [3]:
# 데이터 불러오기 및 전처리 작업
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False)

# Class
#'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'

Files already downloaded and verified
Files already downloaded and verified


### 9.3 Pretrained model 불러오기
파이토치에서는 다양한 사전 학습 된 모델을 제공하고 있다.
https://pytorch.org/docs/stable/torchvision/models.html

In [4]:
# AlexNet 불러오기 
# pretrained=True를 하면 AlexNet 구조와 사전 학습 된 파라메타를 모두 불러온다.
# pretrained=False를 하면 AlexNet 구조만 불러온다.

model = torchvision.models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to C:\Users\신상훈/.cache\torch\hub\checkpoints\alexnet-owt-4df8aa71.pth


In [5]:
# 모델의 구조를 보면 마지막 출력 노드가 1000개라는 것을 알 수 있다. 
# 이는 1000개의 클래스를 가진 ImageNet 데이터를 이용하여 사전학습 된 모델이기 때문이다. 
# 따라서 우리가 사용하는 CIFAR10 데이터에 맞게 출력층의 노드를 10개로 변경해야만 한다.

num_ftrs = model.classifier[6].in_features # fc의 입력 노드 수를 산출한다. 
model.classifier[6] = nn.Linear(num_ftrs, 10) # fc를 nn.Linear(num_ftrs, 10)로 대체한다.
model = model.to(device)

In [6]:
# 출력층의 노드가 10개로 바껴있다.
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## 9.4 모델 프리징

In [7]:
# 파라메타 번호 확인 하기
i = 0
for name, param in model.named_parameters():
    
    print(i,name)
    i+= 1

0 features.0.weight
1 features.0.bias
2 features.3.weight
3 features.3.bias
4 features.6.weight
5 features.6.bias
6 features.8.weight
7 features.8.bias
8 features.10.weight
9 features.10.bias
10 classifier.1.weight
11 classifier.1.bias
12 classifier.4.weight
13 classifier.4.bias
14 classifier.6.weight
15 classifier.6.bias


In [8]:
# 합성곱 층은 0~9까지이다. 따라서 9번째 변수까지 역추적을 비활성화 한 후 for문을 종료한다.

for i, (name, param) in enumerate(model.named_parameters()):
    
    param.requires_grad = False
    if i == 9:
        print('end')
        break

end


In [9]:
# requires_grad 확인
print(model.features[0].weight.requires_grad)
print(model.features[0].bias.requires_grad)
print(model.features[3].weight.requires_grad)
print(model.features[3].bias.requires_grad)
print(model.features[6].weight.requires_grad)
print(model.features[6].bias.requires_grad)
print(model.features[8].weight.requires_grad)
print(model.features[8].bias.requires_grad)
print(model.features[10].weight.requires_grad)
print(model.features[10].bias.requires_grad)
print(model.classifier[1].weight.requires_grad)
print(model.classifier[1].bias.requires_grad)
print(model.classifier[4].weight.requires_grad)
print(model.classifier[4].bias.requires_grad)
print(model.classifier[6].weight.requires_grad)
print(model.classifier[6].bias.requires_grad)

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True


## 9.5 손실함수와 최적화 방법 정의

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-2)

## 9.6 프리징 된 사전학습 모델을 이용한 학습

In [11]:
for epoch in range(20):

    running_loss = 0.0
    for data in trainloader:
        
        inputs, labels = data[0].to(device), data[1].to(device)
          
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    cost = running_loss / len(trainloader)        
    print('[%d] loss: %.3f' %(epoch + 1, cost))  
   

print('Finished Training')

RuntimeError: cuDNN error: CUDNN_STATUS_ALLOC_FAILED

## 9.7 모델 평가

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))